Demo of land cover regridding module.

In [2]:
import xarray as xr
import flox
import numpy as np
import pandas as pd

np.random.seed(10)

In [5]:
lat = np.arange(1,5,0.5)
lat

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])

In [12]:
# dummy land cover dataset
dummy_data = np.random.randint(6, size=(1, 4, 5))
lat = np.arange(1, 3, 0.5)
lon = np.arange(12, 14.5, 0.5)
time = pd.Timestamp("2020-09-01")

In [13]:
ds = xr.Dataset(
    data_vars=dict(
        land_cover=(["time", "lat", "lon"], dummy_data),
    ),
    coords=dict(
        time=[time],
        lat=(["lat"], lat),
        lon=(["lon"], lon),
    ),
    attrs=dict(description="Dummy land cover data."),
)
ds

<xarray.Dataset>
Dimensions:     (time: 1, lat: 4, lon: 5)
Coordinates:
  * time        (time) datetime64[ns] 2020-09-01
  * lat         (lat) float64 1.0 1.5 2.0 2.5
  * lon         (lon) float64 12.0 12.5 13.0 13.5 14.0
Data variables:
    land_cover  (time, lat, lon) int64 0 1 1 0 1 2 4 2 1 1 4 0 0 5 1 0 4 4 0 2
Attributes:
    description:  Dummy land cover data.

In [14]:
ds["land_cover"].values

array([[[0, 1, 1, 0, 1],
        [2, 4, 2, 1, 1],
        [4, 0, 0, 5, 1],
        [0, 4, 4, 0, 2]]])

In [26]:
lat_bounds = pd.IntervalIndex.from_breaks([0, 2, 3], closed="right")
lon_bounds = pd.IntervalIndex.from_breaks([0, 1, 2, 3], closed="right")

In [27]:
from flox import Aggregation
import flox.xarray

# Here is an example of groupby multiple dimensions using flox
#https://flox.readthedocs.io/en/latest/intro.html#histogramming-binning-by-multiple-variables

# To embed our customized function, we need to custom an aggregation
# Here is a list of supported aggregations by flox
# https://flox.readthedocs.io/en/latest/aggregations.html

# To customize the Aggregation function, we need to work with the `numpy_groupies.aggregate_numpy.aggregate` API
# Here is an example about how to implement it
# https://flox.readthedocs.io/en/latest/user-stories/custom-aggregations.html

def most_frequent_label(neighbors):
    unique_labels, counts = np.unique(neighbors, return_counts=True)
    return unique_labels[np.argmax(counts)]

In [28]:
import numpy_groupies as npg

def custom_grouped_reduction(
    group_idx, array, *, axis=-1, size=None, fill_value=None, dtype=None
):
    """
    Parameters
    ----------

    group_idx : np.ndarray, 1D
        integer codes for group labels (1D)
    array : np.ndarray, nD
        values to reduce (nD)
    axis : int
        axis of array along which to reduce. Requires array.shape[axis] == len(group_idx)
    size : int, optional
        expected number of groups. If none, output.shape[-1] == number of uniques in group_idx
    fill_value : optional
        fill_value for when number groups in group_idx is less than size
    dtype : optional
        dtype of output

    Returns
    -------

    np.ndarray with array.shape[-1] == size, containing a single value per group
    """
    return npg.aggregate_numpy.aggregate(
        group_idx,
        array,
        func=most_frequent_label,
        axis=axis,
        size=size,
        fill_value=fill_value,
        dtype=dtype,
    )

most_common = Aggregation(name="most_common", numpy=custom_grouped_reduction, chunk=None, combine=None)

In [30]:
da.values

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [29]:
da_zonal_stat = flox.xarray.xarray_reduce(da, "lat", "lon", func=most_common, expected_groups=(lat_bounds, lon_bounds))
da_zonal_stat

<xarray.DataArray (lat_bins: 2, lon_bins: 3)>
array([[ 0,  1,  3],
       [ 8,  9, 11]])
Coordinates:
  * lat_bins  (lat_bins) object (0, 2] (2, 3]
  * lon_bins  (lon_bins) object (0, 1] (1, 2] (2, 3]

In [6]:
aa = np.arange(12)
bb = np.array([1,2,2,3])

def most_frequent_label(neighbors):
    unique_labels, counts = np.unique(neighbors, return_counts=True)
    return unique_labels[np.argmax(counts)]

most_frequent_label(bb)

2

In [13]:
arr = np.ones((4, 12))
labels1 = np.array(["a", "a", "c", "c", "c", "b", "b", "c", "c", "b", "b", "f"])
labels2 = np.array([1, 2, 2, 1])

da = xr.DataArray(
    arr, dims=("x", "y"), coords={"labels2": ("x", labels2), "labels1": ("y", labels1)}
)
da

<xarray.DataArray (x: 4, y: 12)>
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
Coordinates:
    labels2  (x) int64 1 2 2 1
    labels1  (y) <U1 'a' 'a' 'c' 'c' 'c' 'b' 'b' 'c' 'c' 'b' 'b' 'f'
Dimensions without coordinates: x, y

In [14]:
flox.xarray.xarray_reduce(da, "labels1", "labels2", func="sum")

<xarray.DataArray (labels1: 4, labels2: 2)>
array([[ 4.,  4.],
       [ 8.,  8.],
       [10., 10.],
       [ 2.,  2.]])
Coordinates:
  * labels1  (labels1) object 'a' 'b' 'c' 'f'
  * labels2  (labels2) int64 1 2